In [1]:
import nltk
import numpy as np
from sklearn.preprocessing import normalize
from nltk.corpus import genesis as gen
from nltk import pos_tag
np.random.seed(10000)

In [2]:
word_corpus = {}
word_corpus_rev = {}
num = 0
for word in (nltk.corpus.genesis.words()):
    if word not in word_corpus and word.isalpha():
        word_corpus_rev[num] = word
        word_corpus[word] = num
        num += 1
len(word_corpus_rev)

25513

In [36]:
emission.sum(axis = 1)

array([1., 1., 1., ..., 1., 1., 1.])

In [3]:
def generate_random(a):
    index = np.random.permutation(len(a))[0:10]
    a[index] = np.random.random(10)
    return a

In [35]:
states = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP','RB','RBR','RBS','RP','TO','UH',"VB",'VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP','WRB']
transition = np.zeros((len(states),len(states)))
emission = np.zeros((len(word_corpus),len(states)))
initial = np.zeros((len(states),1))
transition = normalize(np.apply_along_axis(generate_random, 0,transition), norm = 'l1')
emission = normalize(np.apply_along_axis(generate_random, 1,emission), norm = 'l1')
initial = normalize(np.apply_along_axis(generate_random, 0,initial), norm = 'l1')
# transition = normalize(np.random.random((len(states),len(states))), norm='l1') * 100
# emission = normalize(np.random.random((len(word_corpus),len(states))), norm = 'l1') * 100
# initial = normalize(np.random.random((len(states),1)), norm = 'l1')

In [26]:
print(emission.T.shape)

(35, 25513)


In [5]:
transitionc = {}
for i,condition in enumerate(states):
    dic = {}
    for j,state in enumerate(states):
        dic[state] = transition[i][j]
    transitionc[condition] = ( nltk.probability.DictionaryProbDist(dic))
transitionc = nltk.probability.DictionaryConditionalProbDist(transitionc);

emissionc = {}
for i,condition in enumerate(list(word_corpus)):
    dic = {}
    for j,state in enumerate(states):
        dic[state] = emission[i][j]
    emissionc[condition] = ( nltk.probability.DictionaryProbDist(dic))
emissionc = nltk.probability.DictionaryConditionalProbDist(transitionc);

initialc = {}
for i,condition in enumerate(states):
    initialc[condition] = initial[i]
initialc = nltk.probability.DictionaryProbDist(initialc)

In [6]:
model = nltk.tag.hmm.HiddenMarkovModelTagger(list(word_corpus),states,transitionc,emissionc,initialc)

In [7]:
# a = nltk.pos_tag(nltk.corpus.genesis.sents())
obs_nltk = []
obs_blake = []
obs_vidur = []
lens = []
test = []
for sentence in gen.sents():
    sen = []
    sent = []
    for word in sentence:
        if word.isalpha():
#             test.append(word)
            sen.append((word,pos_tag([word])[0][1]))
            obs_blake.append(word_corpus[word])
            sent.append(word_corpus[word])
    lens.append(len(sen))
    obs_nltk.append(sen)
    obs_vidur.append(sent)

In [ ]:
print(len(set(test)),len(set(list(word_corpus))))
print((set(test) - set(word_corpus)))

In [8]:
import time
index = []
obs_vidur = np.array(obs_vidur)
for i, item in enumerate(lens):
    if item < 1:
        print (i)
        index.append(i)
lens = np.delete(lens,index)
obs_vidur = np.delete(obs_vidur,index)

5440
8132
9135
9148
9688
10489
13442


In [9]:
a = []
for sentence in gen.sents():
    sen = []
    for word in sentence:
        if word.isalpha():
            sen.append((word,pos_tag([word])[0][1]))
    a.append(sen)

In [ ]:
trainer = nltk.tag.hmm.HiddenMarkovModelTrainer()
start = time.clock()
trainer.train_unsupervised(unlabeled_sequences=obs_nltk , model = model, max_iterations = 1)
print (time.clock() - start)

In [38]:
%load_ext autoreload
%autoreload 2
from HMM import *
model2 = hmm(len(states),25513)
model2.custom(initial,transition,emission.T)
start = time.clock()
for i in range(2):
    model2.fit(obs_vidur,lens)
time.clock() - start

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/vidur/Desktop/ABT/comparison/HMM.py:183: RuntimeWarning: invalid value encountered in true_divide
  num_a += na/prob
/home/vidur/Desktop/ABT/comparison/HMM.py:184: RuntimeWarning: invalid value encountered in true_divide
  num_b += nb/prob
/home/vidur/Desktop/ABT/comparison/HMM.py:185: RuntimeWarning: invalid value encountered in true_divide
  den_a += da/prob
/home/vidur/Desktop/ABT/comparison/HMM.py:186: RuntimeWarning: invalid value encountered in true_divide
  den_b += db/prob


20.635899771632975
nan


671.937175

In [ ]:
range(5)[4]